In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from src.evaluation import *
from src.utils import load_fold, assets, non_train_cols
from src.models import BaseCryptoLearner
plt.rcParams["figure.figsize"] = (16,9)

In [7]:
import lightgbm as lgb

In [4]:
asset_info = pd.read_csv('../data/gresearch/raw/asset_details.csv')

In [5]:
# Load training data 
train_dir = '../data/gresearch/processed/fold_1/train/'
test_dir = '../data/gresearch/processed/fold_1/test/'

In [6]:
train_df = load_fold(train_dir)
test_df = load_fold(test_dir)

In [21]:
feature_cols = ['direct_return', 'log_return', 'high_low_ratio','rel_change_Target_1min']
target_col = ['Target']

In [23]:
lgb_train_data = lgb.Dataset(train_df[feature_cols],
                             label=train_df[target_col],
                             feature_name=feature_cols)

In [25]:
param = {'objective': 'regression', 'metric': 'mae'}
num_round = 10

bst = lgb.train(param, lgb_train_data, num_round)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1084666, number of used features: 4
[LightGBM] [Info] Start training from score -0.000005


In [29]:
pred = pd.DataFrame({'Asset_ID': train_df.Asset_ID,
                     'timestamp': train_df.timestamp})

pred['prediction'] = bst.predict(train_df[feature_cols])


test = train_df[['Target', 'Weight', 'timestamp', 'Asset_ID']].merge(
    pred, on=['timestamp', 'Asset_ID'], how='left').fillna(0)

score = corr_score(test.Target, test.prediction, test.Weight)

In sample score

In [30]:
score

0.07975201013848894

In [31]:
# Out of sample
pred = pd.DataFrame({'Asset_ID': test_df.Asset_ID,
                     'timestamp': test_df.timestamp})

pred['prediction'] = bst.predict(test_df[feature_cols])


test = test_df[['Target', 'Weight', 'timestamp', 'Asset_ID']].merge(
    pred, on=['timestamp', 'Asset_ID'], how='left').fillna(0)

score = corr_score(test.Target, test.prediction, test.Weight)
print(score)

0.022044524038922934
